In [ ]:
pip install transformers

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('../data/data.csv')

In [ ]:
import torch
print(torch.__version__)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Carregar o tokenizer e o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')


In [ ]:
def get_bert_embeddings(text):
    # Tokenizar o texto e obter tensores
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    
    # Extrair o embedding da última camada
    last_hidden_state = outputs.last_hidden_state
    # Calcular a média dos embeddings de cada palavra para representar o texto
    embeddings = torch.mean(last_hidden_state, dim=1).squeeze()
    
    return embeddings.detach().numpy()


In [ ]:
data['bert_embeddings'] = data['detalhamento'].apply(get_bert_embeddings)

## Testando acurácia de modelo usando BERT

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    data['bert_embeddings'].tolist(), data['resultado'], test_size=0.2, random_state=42
)

# Treinar um classificador usando os embeddings do BERT
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred = clf.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))


## Analisando sentimentos

In [ ]:
from transformers import pipeline
import pandas as pd

# Carregar o pipeline de classificação de sentimentos com BERT
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Exemplo de como carregar seu DataFrame (substitua com o caminho do seu arquivo, se necessário)
df = pd.read_csv('../data/data.csv')

# Aplicar a classificação de sentimento na coluna 'detalhamento' do DataFrame
def classify_sentiment(text):
    result = classifier(text[:512])[0]  # Limitar a 512 caracteres devido à limitação do BERT
    return result['label'], result['score']

# Criar novas colunas no DataFrame para armazenar o sentimento e a pontuação
df[['sentimento', 'confiança']] = df['detalhamento'].apply(lambda x: pd.Series(classify_sentiment(x)))

# Exibir o DataFrame com as novas colunas
print(df[['detalhamento', 'sentimento', 'confiança']].head())


In [ ]:
# Configurar o Pandas para exibir 50 linhas
pd.set_option('display.max_rows', 50)

# Exibir o DataFrame com os resultados de sentimento
print(df[['detalhamento', 'sentimento', 'confiança']])


In [ ]:
# Contar a quantidade de cada rótulo de sentimento
sentiment_counts = df['sentimento'].value_counts()

# Exibir os resultados
print("Contagem dos sentimentos:")
print(sentiment_counts)
